In [13]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Strategy
from backtesting import Backtest


## Strategy

In [2]:
# Fetch the data of META
ticker = 'META'
tickerData = yf.Ticker(ticker)
endDate = datetime.now()
startDate = endDate - timedelta(weeks=104)
df = tickerData.history(period='1d', start=startDate, end=endDate)
meta_df = df[['Open','High','Low','Close','Volume']]
meta_df.reset_index()
meta_df

,Open,High,Low,Close,Volume
Date,,,,,
2021-08-04 00:00:00-04:00,352.420013,360.480011,351.510010,358.920013,14180600
2021-08-05 00:00:00-04:00,359.640015,363.899994,356.899994,362.970001,10247200
2021-08-06 00:00:00-04:00,361.399994,365.149994,361.399994,363.510010,8925000
2021-08-09 00:00:00-04:00,363.760010,365.779999,360.750000,361.609985,7798900
2021-08-10 00:00:00-04:00,361.829987,364.100006,359.040009,361.130005,7469900
...,...,...,...,...,...
2023-07-26 00:00:00-04:00,301.190002,301.769989,291.899994,298.570007,47256900
2023-07-27 00:00:00-04:00,325.119995,325.350006,309.839996,311.709991,64229200
2023-07-28 00:00:00-04:00,316.880005,326.200012,314.250000,325.480011,39160700


In [3]:
# Add indicators of SMA, RSI and Bollinger Bands
meta_df["SMA"] = ta.sma(meta_df.Close, length=200)
meta_df["RSI"] = ta.rsi(meta_df.Close, length=2)
my_bbands = ta.bbands(meta_df.Close, length=20, std=2.5)
# Need to improve the following method to collect bollinger bands
meta_df[["BBL_20_2.5","BBM_20_2.5","BBU_20_2.5","BBB_20_2.5","BBP_20_2.5"]] = my_bbands 
meta_df.dropna()

C:\Users\Yichao Hou\AppData\Local\Temp\ipykernel_14160\1745453865.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_df["SMA"] = ta.sma(meta_df.Close, length=200)


,Open,High,Low,Close,Volume,SMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5
Date,,,,,,,,,,,,
2022-05-18 00:00:00-04:00,200.000000,201.000000,191.630005,192.240005,23960000,292.301551,20.419284,168.638574,197.362501,226.086427,29.107785,0.410832
2022-05-19 00:00:00-04:00,191.199997,195.289993,189.600006,191.289993,24446900,291.463401,17.904556,169.072824,197.523500,225.974176,28.807384,0.390451
2022-05-20 00:00:00-04:00,194.970001,197.910004,187.869995,193.539993,31465600,290.616251,48.150999,170.485278,197.995000,225.504721,27.788299,0.419029
2022-05-23 00:00:00-04:00,195.190002,196.990005,191.240005,196.229996,25059200,289.779851,72.434797,171.650705,198.456999,225.263294,27.014713,0.458461
2022-05-24 00:00:00-04:00,180.550003,183.679993,176.110001,181.279999,50406300,288.878201,11.671951,171.734036,198.473499,225.212963,26.945122,0.178500
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-26 00:00:00-04:00,301.190002,301.769989,291.899994,298.570007,47256900,189.029050,69.082793,272.599122,298.041499,323.483877,17.073044,0.510386
2023-07-27 00:00:00-04:00,325.119995,325.350006,309.839996,311.709991,64229200,189.920350,92.600933,273.985756,299.362498,324.739241,16.953856,0.743284
2023-07-28 00:00:00-04:00,316.880005,326.200012,314.250000,325.480011,39160700,190.878800,97.147962,274.586173,301.559999,328.533825,17.889525,0.943393


In [4]:
final_df = meta_df.dropna()
final_df.shape

(302, 12)

In [5]:
def add_sma_signal(df, backcandles):
    sma_signal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1     # uptrend
        dnt = 1     # downtrend
        # backcandles are continous candles we want to compare with sma line
        # to get the trend of the stock price
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.SMA[i]: # if highest price of candle is larger than SMA, it's an uptrend
                dnt=0
            if df.Low[i]<=df.SMA[i]: # if lowest price of candle is smaller than SMA, it's a downtrend
                upt=0
        if upt==1 and dnt==1:
            sma_signal[row]=3 # Hold the Stok
        elif upt==1:
            sma_signal[row]=2 # Buying Signal
        elif dnt==1:
            sma_signal[row]=1 # Selling Signal
    df['SMA_Signal'] = sma_signal

add_sma_signal(final_df, 3)

C:\Users\Yichao Hou\AppData\Local\Temp\ipykernel_14160\2180050507.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA_Signal'] = sma_signal


In [6]:
final_df.head()

,Open,High,Low,Close,Volume,SMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,SMA_Signal
Date,,,,,,,,,,,,,
2022-05-18 00:00:00-04:00,200.000000,201.000000,191.630005,192.240005,23960000,292.301551,20.419284,168.638574,197.362501,226.086427,29.107785,0.410832,0
2022-05-19 00:00:00-04:00,191.199997,195.289993,189.600006,191.289993,24446900,291.463401,17.904556,169.072824,197.523500,225.974176,28.807384,0.390451,0
2022-05-20 00:00:00-04:00,194.970001,197.910004,187.869995,193.539993,31465600,290.616251,48.150999,170.485278,197.995000,225.504721,27.788299,0.419029,0
2022-05-23 00:00:00-04:00,195.190002,196.990005,191.240005,196.229996,25059200,289.779851,72.434797,171.650705,198.456999,225.263294,27.014713,0.458461,1
2022-05-24 00:00:00-04:00,180.550003,183.679993,176.110001,181.279999,50406300,288.878201,11.671951,171.734036,198.473499,225.212963,26.945122,0.178500,1


In [7]:
def add_orders_limit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if df.SMA_Signal[i]==2 and df.Close[i]<=df['BBL_20_2.5'][i]:# and df.RSI[i]<=100: #Added RSI condition to avoid direct close condition
            ordersignal[i]=df.Close[i]-df.Close[i]*percent  # The way to get limit price
        elif df.SMA_Signal[i]==1 and df.Close[i]>=df['BBU_20_2.5'][i]:# and df.RSI[i]>=0:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent      # The way to get limit price
    df['ordersignal']=ordersignal

add_orders_limit(final_df, 0.01)

C:\Users\Yichao Hou\AppData\Local\Temp\ipykernel_14160\380323091.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ordersignal']=ordersignal


In [8]:
final_df.tail(20)

,Open,High,Low,Close,Volume,SMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,SMA_Signal,ordersignal
Date,,,,,,,,,,,,,,
2023-07-05 00:00:00-04:00,287.649994,298.119995,286.359985,294.369995,33865500,176.931650,89.107112,257.487356,279.104500,300.721644,15.490359,0.853088,2,0.0
2023-07-06 00:00:00-04:00,295.890015,298.119995,291.309998,291.989990,47600500,177.660150,62.622817,257.956583,280.148000,302.339416,15.842638,0.766815,2,0.0
2023-07-07 00:00:00-04:00,292.179993,296.200012,288.660004,290.529999,25546200,178.372700,45.889224,260.776472,281.494499,302.212526,14.720023,0.718059,2,0.0
2023-07-10 00:00:00-04:00,295.549988,298.130005,287.049988,294.100006,37058300,179.112750,76.542813,263.583135,282.970500,302.357865,13.702746,0.787030,2,0.0
2023-07-11 00:00:00-04:00,293.899994,300.179993,291.899994,298.290009,27585900,179.893600,89.931494,266.461663,284.637500,302.813337,12.771217,0.875568,2,0.0
2023-07-12 00:00:00-04:00,301.750000,309.450012,300.100006,309.339996,36677100,180.726200,97.489467,265.565085,286.552000,307.538915,14.647893,1.042910,2,0.0
2023-07-13 00:00:00-04:00,313.619995,316.239990,310.290009,313.410004,30281000,181.591200,98.383402,264.872299,288.656500,312.440701,16.479241,1.020377,2,0.0
2023-07-14 00:00:00-04:00,311.790009,314.880005,307.359985,308.869995,22576000,182.453700,54.828463,265.928575,290.432500,314.936424,16.874093,0.876215,2,0.0
2023-07-17 00:00:00-04:00,307.540009,311.709991,304.709991,310.619995,25323100,183.334800,66.322385,265.571430,291.872000,318.172570,18.021989,0.856418,2,0.0


In [9]:
final_df[final_df.ordersignal != 0]

,Open,High,Low,Close,Volume,SMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,SMA_Signal,ordersignal
Date,,,,,,,,,,,,,,
2022-07-20 00:00:00-04:00,177.490005,183.600006,176.699997,183.089996,24784300,252.702201,96.969514,149.834166,165.983501,182.132837,19.458965,1.029635,1,184.920896
2023-01-04 00:00:00-05:00,127.379997,129.050003,125.849998,127.370003,32397100,159.852200,97.468881,109.412548,118.150500,126.888453,14.791225,1.027555,1,128.643703


## Visualization

In [10]:
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
    
final_df['pointposbreak'] = final_df.apply(lambda row: pointposbreak(row), axis=1)

C:\Users\Yichao Hou\AppData\Local\Temp\ipykernel_14160\2870692633.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['pointposbreak'] = final_df.apply(lambda row: pointposbreak(row), axis=1)


In [11]:
dfpl = final_df.copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.SMA, line=dict(color='orange', width=2), name="SMA"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.5'], line=dict(color='blue', width=1), name="BBL_20_2.5"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.5'], line=dict(color='blue', width=1), name="BBU_20_2.5")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [21]:
dfpl.index

DatetimeIndex(['2022-05-18 00:00:00-04:00', '2022-05-19 00:00:00-04:00',
               '2022-05-20 00:00:00-04:00', '2022-05-23 00:00:00-04:00',
               '2022-05-24 00:00:00-04:00', '2022-05-25 00:00:00-04:00',
               '2022-05-26 00:00:00-04:00', '2022-05-27 00:00:00-04:00',
               '2022-05-31 00:00:00-04:00', '2022-06-01 00:00:00-04:00',
               ...
               '2023-07-19 00:00:00-04:00', '2023-07-20 00:00:00-04:00',
               '2023-07-21 00:00:00-04:00', '2023-07-24 00:00:00-04:00',
               '2023-07-25 00:00:00-04:00', '2023-07-26 00:00:00-04:00',
               '2023-07-27 00:00:00-04:00', '2023-07-28 00:00:00-04:00',
               '2023-07-31 00:00:00-04:00', '2023-08-01 00:00:00-04:00'],
              dtype='datetime64[ns, America/New_York]', name='Date', length=302, freq=None)

## Backtesting

In [12]:
def SIGNAL():
    return dfpl.ordersignal

In [22]:
class MyStrat(Strategy):
    initsize = 0.05
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()

        for j in range(0, len(self.orders)):
            if (datetime.strptime(self.data.index[-1], '%Y-%m-%d %H:%M:%S')-self.ordertime[0]) > 10:#days max to fulfill the order!!!
                self.orders[0].cancel()
                self.ordertime.pop(0)

        if len(self.trades)>0:
            if (self.data.index[-1]-self.trades[-1].entry_time) >= 10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)

            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()

        if self.signal!=0 and len(self.trades)==0 and self.data.SMA_Signal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

        elif self.signal!=0 and len(self.trades)==0 and self.data.SMA_Signal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])


In [23]:
bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/100, commission=.000)
stat = bt.run()
stat

TypeError: strptime() takes exactly 2 arguments (1 given)